In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import calendar

import xarray as xr

FAOSTAT food balance sheets can be use to describe dietary patterns and estimate their impact and the impact of interventions on multiple indicators derived from it.
Latest FAOSTAT data can be downloaded from the FAO website:
- https://www.fao.org/faostat/en/#data/FBSH for data released until 2013 utilizing old methodology
- https://www.fao.org/faostat/en/#data/FBS for data published since 2010 utilizing new methodology

For both datasets, select the 'All Data Normalized' option on the right hand side panel of the sites to get two compressed folders which include the two files named below.

In this notebook the values are extracted and stored in an easy to manage n-dimensional numpy array which is then read by package when the FAOSTAT dataset is imported.

The dimensions of the array include:

- Year
- Region
- Element
- Item

In [2]:
# Read historic and current data
fbs_csv_1 = '../../data/food/FoodBalanceSheetsHistoric_E_All_Data_(Normalized).csv'
fbs_csv_2 = '../../data/food/FoodBalanceSheets_E_All_Data_(Normalized).csv'

In [3]:
fbs_1 = pd.read_csv(fbs_csv_1, encoding = 'latin1') # 1961 - 2013
fbs_2 = pd.read_csv(fbs_csv_2, encoding = 'latin1') # 2010 - present

# We will prefer new methodology over old, so will drop 2010-2013 year range from old data
fbs_1 = fbs_1[fbs_1['Year'].isin(np.arange(1961, 2010))]

fbs = pd.concat([fbs_2, fbs_1], ignore_index=True, sort=False)
# fbs = fbs[fbs['Area Code'] == 229]

fbs = fbs[['Area Code', 'Area', 'Element', 'Element Code', 'Item', 'Item Code', 'Year', 'Unit', 'Value']]
years = np.unique(fbs['Year'])

'Rice and products' and 'Groundnuts' changed name and code from one methodology to another. We will keep the new codes and names

'Miscellaneous', 'Milk - Excluding Butter' and 'Eggs' have two different measurement flags, each with a different 'Item Code'. Will keep the highest Item Code: 2948 & 2949 for the later two.

In [4]:
nuts_old_code = fbs_1[fbs_1['Item'] == 'Groundnuts (Shelled Eq)'].iloc[0]['Item Code']
rice_old_code = fbs_1[fbs_1['Item'] == 'Rice (Milled Equivalent)'].iloc[0]['Item Code']

nuts_new_code = fbs_2[fbs_2['Item'] == 'Groundnuts'].iloc[0]['Item Code']
rice_new_code = fbs_2[fbs_2['Item'] == 'Rice and products'].iloc[0]['Item Code']

milk_codes = pd.unique(fbs[fbs['Item'] == 'Milk - Excluding Butter']['Item Code'])
eggs_codes = pd.unique(fbs[fbs['Item'] == 'Eggs']['Item Code'])


print(f'Groundnuts: old code = {nuts_old_code}, new code = {nuts_new_code}')
print(f'Rice and products: old code = {rice_old_code}, new code = {rice_new_code}')

print('Milk codes:', milk_codes)
print('Eggs codes:', eggs_codes)

Groundnuts: old code = 2556, new code = 2552
Rice and products: old code = 2805, new code = 2807
Milk codes: [2948 2848]
Eggs codes: [2949 2744]


In [5]:
# We'll rename some of the items due to changes in naming convention
fbs.loc[fbs['Item Code'] == rice_old_code, 'Item'] = "Rice and products"
fbs.loc[fbs['Item Code'] == rice_old_code, 'Item Code'] = rice_new_code

fbs.loc[fbs['Item Code'] == nuts_old_code, 'Item'] = "Groundnuts"
fbs.loc[fbs['Item Code'] == nuts_old_code, 'Item Code'] = nuts_new_code

# Also, will remove the 'Miscellaneous' item as it has incomplete data
fbs.drop(fbs.loc[fbs['Item'] == 'Miscellaneous'].index, inplace=True)

# Also, will remove the 'Miscellaneous' item as it has incomplete data
fbs.drop(fbs.loc[fbs['Item'] == 'Grand Total'].index, inplace=True)

# Finally, drop all duplicates of Eggs and Milk and population data
fbs.drop(fbs.loc[fbs['Item Code'].isin([milk_codes[1], eggs_codes[1], 2501])].index, inplace=True)

fbs

,Area Code,Area,Element,Element Code,Item,Item Code,Year,Unit,Value
50,2,Afghanistan,Food supply (kcal/capita/day),664,Vegetal Products,2903,2010,kcal/capita/day,1964.00
51,2,Afghanistan,Food supply (kcal/capita/day),664,Vegetal Products,2903,2011,kcal/capita/day,1953.00
52,2,Afghanistan,Food supply (kcal/capita/day),664,Vegetal Products,2903,2012,kcal/capita/day,1955.00
53,2,Afghanistan,Food supply (kcal/capita/day),664,Vegetal Products,2903,2013,kcal/capita/day,1993.00
54,2,Afghanistan,Food supply (kcal/capita/day),664,Vegetal Products,2903,2014,kcal/capita/day,2019.00
...,...,...,...,...,...,...,...,...,...
13533079,5817,Net Food Importing Developing Countries,Fat supply quantity (g/capita/day),684,Infant food,2680,2005,g/capita/day,0.00
13533080,5817,Net Food Importing Developing Countries,Fat supply quantity (g/capita/day),684,Infant food,2680,2006,g/capita/day,0.00
13533081,5817,Net Food Importing Developing Countries,Fat supply quantity (g/capita/day),684,Infant food,2680,2007,g/capita/day,0.01
13533082,5817,Net Food Importing Developing Countries,Fat supply quantity (g/capita/day),684,Infant food,2680,2008,g/capita/day,0.01


Food items are identified by the `"Item Code"` column.

In [6]:
area_names = pd.unique(fbs['Area'])
area_codes = pd.unique(fbs['Area Code'])

sorting = np.argsort(area_codes)
area_codes = area_codes[sorting]
area_names = area_names[sorting]

for name, code in zip(area_names, area_codes):
    print(code, name)

1 Armenia
2 Afghanistan
3 Albania
4 Algeria
7 Angola
8 Antigua and Barbuda
9 Argentina
10 Australia
11 Austria
12 Bahamas
14 Barbados
15 Belgium-Luxembourg
16 Bangladesh
17 Bermuda
19 Bolivia (Plurinational State of)
20 Botswana
21 Brazil
23 Belize
25 Solomon Islands
26 Brunei Darussalam
27 Bulgaria
28 Myanmar
29 Burundi
32 Cameroon
33 Canada
35 Cabo Verde
37 Central African Republic
38 Sri Lanka
39 Chad
40 Chile
41 China, mainland
44 Colombia
45 Comoros
46 Congo
48 Costa Rica
49 Cuba
50 Cyprus
51 Czechoslovakia
52 Azerbaijan
53 Benin
54 Denmark
55 Dominica
56 Dominican Republic
57 Belarus
58 Ecuador
59 Egypt
60 El Salvador
62 Ethiopia PDR
63 Estonia
66 Fiji
67 Finland
68 France
70 French Polynesia
72 Djibouti
73 Georgia
74 Gabon
75 Gambia
79 Germany
80 Bosnia and Herzegovina
81 Ghana
83 Kiribati
84 Greece
86 Grenada
89 Guatemala
90 Guinea
91 Guyana
93 Haiti
95 Honduras
96 China, Hong Kong SAR
97 Hungary
98 Croatia
99 Iceland
100 India
101 Indonesia
102 Iran (Islamic Republic of)
103 I

In [7]:
item_names = pd.unique(fbs['Item'])
item_codes = pd.unique(fbs['Item Code'])

# Sort by item code
sorting = np.argsort(item_codes)
item_codes = item_codes[sorting]
item_names = item_names[sorting]

for name, code in zip(item_names, item_codes):
    print(code, name)

2511 Wheat and products
2513 Barley and products
2514 Maize and products
2515 Rye and products
2516 Oats
2517 Millet and products
2518 Sorghum and products
2520 Cereals, Other
2531 Potatoes and products
2532 Cassava and products
2533 Sweet potatoes
2534 Roots, Other
2535 Yams
2536 Sugar cane
2537 Sugar beet
2541 Sugar non-centrifugal
2542 Sugar (Raw Equivalent)
2543 Sweeteners, Other
2546 Beans
2547 Peas
2549 Pulses, Other and products
2551 Nuts and products
2552 Groundnuts
2555 Soyabeans
2557 Sunflower seed
2558 Rape and Mustardseed
2559 Cottonseed
2560 Coconuts - Incl Copra
2561 Sesame seed
2562 Palm kernels
2563 Olives (including preserved)
2570 Oilcrops, Other
2571 Soyabean Oil
2572 Groundnut Oil
2573 Sunflowerseed Oil
2574 Rape and Mustard Oil
2575 Cottonseed Oil
2576 Palmkernel Oil
2577 Palm Oil
2578 Coconut Oil
2579 Sesameseed Oil
2580 Olive Oil
2581 Ricebran Oil
2582 Maize Germ Oil
2586 Oilcrops Oil, Other
2601 Tomatoes and products
2602 Onions
2605 Vegetables, other
2611 Orang

Different information and values are encoded by the `"Element code"`:

In [8]:
# Element codes, units and names. Remove population data
element_codes = pd.unique(fbs['Element Code'])
element_names = pd.unique(fbs['Element'])
element_units= []

for code in element_codes:
    unit = fbs[fbs['Element Code'] == code].iloc[0]['Unit']
    element_units.append(unit)

# Sort by item code
sorting = np.argsort(element_codes)
element_codes = element_codes[sorting]
element_names = element_names[sorting]
    
for i, (code, name) in enumerate(zip(element_codes, element_names)):
    print(code, element_units[i], name )

645 kcal/capita/day Food supply quantity (kg/capita/yr)
664 g/capita/day Food supply (kcal/capita/day)
674 g/capita/day Protein supply quantity (g/capita/day)
684 1000 tonnes Fat supply quantity (g/capita/day)
5072 1000 tonnes Stock Variation
5123 1000 tonnes Losses
5131 1000 tonnes Processing
5142 1000 tonnes Food
5154 1000 tonnes Other uses (non-food)
5170 1000 tonnes Residuals
5171 1000 tonnes Tourist consumption
5301 1000 tonnes Domestic supply quantity
5511 1000 tonnes Production
5521 1000 tonnes Feed
5527 1000 tonnes Seed
5611 1000 tonnes Import Quantity
5911 kg Export Quantity


# Filling incomplete data and store into array

Checking for incomplete data shows that several Items are not present in all areas, and some items without certain nutrients are ommited from the data.
We will fix that to have a rectangular array to facilitate usage with xarray. When data is not present, we will fill with `np.NaN`

In [9]:
# We define indexes for each of the dimensions of the data
iy = fbs['Year'] - 1961

# Some numpy trickery to get the indices for the area, elements and items
ia = np.nonzero(fbs['Area Code'].to_numpy()[:, None] == area_codes)[1]
ie = np.nonzero(fbs['Element Code'].to_numpy()[:, None] == element_codes)[1]
ic = np.nonzero(fbs['Item Code'].to_numpy()[:, None] == item_codes)[1]

# We compute the index in the flattened array
ipos = ia * len(years) * len(element_codes) * len(item_codes) + iy * len(element_codes) * len(item_codes) + ie * len(item_codes) + ic
ipos = ipos.astype(int)

In [10]:
fbs_arr = np.ones(len(area_codes)*len(years)*len(element_codes)*len(item_codes))*np.NaN
fbs_arr[ipos] = fbs['Value']

fbs_arr = fbs_arr.reshape((len(area_codes), len(years), len(element_codes), len(item_codes)))

In [11]:
# Building the xarray dataset
coords = {"Region":area_codes,
          "Year":years,
          "Element":element_codes,
          "Item":item_codes}

data = xr.DataArray(fbs_arr, dims = ("Region", "Year", "Element", "Item"), coords = coords)
data = data.astype(np.float32) # np.float64?

Checking that the arrays are indexed correctly

In [12]:
print(fbs[(fbs["Area Code"]==229) &
          (fbs["Year"]==2019) &
          (fbs["Element Code"]==645) &
          (fbs["Item Code"]== 2511)]['Value'].values)

print(data.sel(Region=229, 
               Year=2019, 
               Element=645, 
               Item=2511).values)

[111.17]
111.17


In [13]:
data.to_netcdf("../../pyourfood/food/data/FAOSTAT.nc")